<a href="https://colab.research.google.com/github/ppiont/cnn-soc-wagga/blob/master/cnn_colab_new.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [33]:
pip install scikit-optimize

In [34]:
from google.colab import drive
drive.mount('/content/drive/')
%cd "/content/drive/MyDrive/Thesis/cnn-soc-wagga"

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).
/content/drive/MyDrive/Thesis/cnn-soc-wagga


In [35]:
# Standard lib imports
import os
import pathlib
import random

# Imports
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import KFold
from sklearn.linear_model import LinearRegression
from torch.utils.data import Dataset, DataLoader
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder, RobustScaler, FunctionTransformer
from sklearn.base import BaseEstimator, TransformerMixin
from skopt import gp_minimize
from skopt.space import Integer, Real, Categorical
from skopt.utils import use_named_args
from skopt.plots import plot_objective, plot_convergence, plot_evaluations
import torch
import torch.nn as nn
import torch.optim as optim

# import torch.nn.functional as F
from torch.utils.data import DataLoader
import copy

# Custom imports
# from feat_eng.funcs import add_min, safe_log, get_corr_feats, min_max
from custom_metrics.metrics import mean_error, lin_ccc, model_efficiency_coefficient

In [36]:
# ------------------- TO DO ------------------------------------------------- #

"""
Use Torch Dataset.. you made a class for it dummy
"""

# ------------------- Settings ---------------------------------------------- #


# Set matploblib style
plt.style.use('seaborn-colorblind')
colors = plt.rcParams['axes.prop_cycle'].by_key()['color']
plt.rcParams['figure.dpi'] = 450
plt.rcParams['savefig.transparent'] = True
plt.rcParams['savefig.format'] = 'svg'

# Reset params if needed
# plt.rcParams.update(mpl.rcParamsDefault)


# ------------------- Organization ------------------------------------------ #


DATA_DIR = pathlib.Path('data/')


def seed_everything(SEED=43):
    random.seed(SEED)
    np.random.seed(SEED)
    torch.manual_seed(SEED)
    torch.cuda.manual_seed(SEED)
    torch.cuda.manual_seed_all(SEED)
    torch.backends.cudnn.deterministic = True
    os.environ['PYTHONHASHSEED'] = str(SEED)
    torch.backends.cudnn.benchmark = False


SEED = 43
seed_everything(SEED=SEED)


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
torch.set_default_tensor_type('torch.cuda.FloatTensor')

In [37]:
# ------------------- Read and prep data ------------------------------------ #


class Dataset(torch.utils.data.TensorDataset):
    """Characterize a PyTorch Dataset."""

    def __init__(self, features, targets):
        super().__init__()

        """Initialize with X and y."""
        self.features = torch.from_numpy(features).permute(0, 3, 1, 2)
        self.targets = torch.from_numpy(targets)

    def __len__(self):
        """Return total number of samples."""
        return len(self.targets)

    def __getitem__(self, idx):
        """Generate one data sample."""

        if torch.is_tensor(idx):
            idx = idx.tolist()

        return self.features[idx].to(device), self.targets[idx].to(device)

features = np.load("data/cnn_features.npy")
targets = np.load("data/cnn_targets.npy")[:, 0]


# Train-val-test split 7-2-1
x_train_, x_test, y_train_, y_test = train_test_split(features, targets, test_size=2 / 10)
x_train, x_val, y_train, y_val = train_test_split(x_train_, y_train_, test_size=2 / 8)

feature_reshaper = FunctionTransformer(
    func=np.reshape,
    inverse_func=np.reshape,
    kw_args={"newshape": (-1, 43)},
    inv_kw_args={"newshape": (-1, 15, 15, 43)},
)
feature_inverse_reshaper = FunctionTransformer(func=np.reshape, kw_args={"newshape": (-1, 15, 15, 43)})

target_reshaper = FunctionTransformer(func=np.reshape, kw_args={"newshape": (-1, 1)})

# Preprocessing
feature_transformer = Pipeline(
    steps=[
        ("reshaper", feature_reshaper),
        ("minmax_scaler", MinMaxScaler()),
        ("inverse_reshaper", feature_inverse_reshaper),
    ]
)
target_transformer = Pipeline(steps=[("reshaper", target_reshaper), ("minmax_scaler", MinMaxScaler())])

kfold_data = Dataset(feature_transformer.fit_transform(x_train_), target_transformer.fit_transform(y_train_))
# train_data = Dataset(feature_transformer.fit_transform(x_train), target_transformer.fit_transform(y_train))
# val_data = Dataset(feature_transformer.transform(x_val), target_transformer.transform(y_val))
# test_data = Dataset(feature_transformer.transform(x_test), target_transformer.transform(y_test))

# batch_size = 128

# train_loader = DataLoader(train_data, batch_size=batch_size, num_workers=2, pin_memory=False)
# val_loader = DataLoader(val_data, batch_size=batch_size, num_workers=2, pin_memory=False)
# test_loader = DataLoader(test_data, batch_size=batch_size, num_workers=2, pin_memory=False)

In [38]:
# ------------------ CNN setup ---------------------------------------------- #


class CNN(nn.Module):
    """Neural Network class."""

    def __init__(self, conv1_channels=32, conv2_channels=64, linear1_neurons=64, linear2_neurons=32):
        """Initialize as subclass of nn.Module, inherit its methods."""
        super().__init__()

        self.conv1_channels = conv1_channels
        self.conv2_channels = conv2_channels
        self.linear1_neurons = linear1_neurons
        self.linear2_neurons = linear2_neurons

        self.conv1 = nn.Conv2d(43, self.conv1_channels, kernel_size=3, stride=3)  # stride i stedet for maxpool
        self.relu = nn.ReLU()
        self.bn1 = nn.BatchNorm2d(self.conv1_channels)
        self.conv2 = nn.Conv2d(self.conv1_channels, self.conv2_channels, 3)
        self.bn2 = nn.BatchNorm2d(self.conv2_channels)
        self.flat = nn.Flatten()
        self.fc1 = nn.LazyLinear(self.linear1_neurons)
        self.bn3 = nn.BatchNorm1d(self.linear1_neurons)
        self.fc2 = nn.Linear(self.linear1_neurons, self.linear2_neurons)
        self.bn4 = nn.BatchNorm1d(self.linear2_neurons)
        self.out = nn.Linear(self.linear2_neurons, 1)


    def forward(self, x):
        x.to(device)
        x = self.conv1(x)
        x = self.relu(x)
        x = self.bn1(x)
        x = self.conv2(x)
        x = self.relu(x)
        x = self.bn2(x)
        x = self.flat(x)
        x = self.fc1(x)
        x = self.relu(x)
        x = self.bn3(x)
        x = self.fc2(x)
        x = self.bn4(x)
        x = self.out(x)

        return x  # prediction


def train_step(model, features, targets, optimizer, loss_fn):
    """Perform a single training step.

    Calulcates prediction, loss and gradients for a single batch
    and updates optimizer parameters accordingly."""

    # Set gradients to zero
    model.zero_grad()
    # Pass data through model
    output = model(features)
    # Calculate loss
    loss = loss_fn(output, targets)
    # Calculate gradients
    loss.backward()
    # Update parameters
    optimizer.step()

    return loss, output


def train_network(model, train_data, val_data, optimizer, loss_fn, n_epochs=2000, patience=100, print_progress=True):
    """Train a neural network model."""
    # Initalize loss as very high
    best_loss = 1e8

    # Create lists to hold train and val losses
    train_loss = []
    val_loss = []
    # Init epochs_no_improve
    epochs_no_improve = 0
    # best_model = copy.deepcopy(model.state_dict())

    # Start training (loop over epochs)
    for epoch in range(n_epochs):

        # Initalize epoch train loss
        train_epoch_loss = 0
        # Loop over training batches
        model.train()  # set model to training mode for training
        for bidx, (features, targets) in enumerate(train_data):
            # Calculate loss and predictions
            loss, predictions = train_step(model, features, targets, optimizer, loss_fn)
            train_epoch_loss += loss
        # Save train epoch loss
        train_loss.append(train_epoch_loss.item())

        # Initialize val epoch loss
        val_epoch_loss = 0
        # Loop over validation batches
        model.eval()  # set model to evaluation mode for validation
        for bidx, (features, targets) in enumerate(val_data):
            output = model(features)
            val_epoch_loss += loss_fn(output, targets)
        # Save val epoch loss
        val_loss.append(val_epoch_loss.item())

        # Early stopping (check if val loss is an improvement on current best)
        if val_epoch_loss < best_loss:
            best_loss = val_epoch_loss.item()
            best_model = copy.deepcopy(model.state_dict())
            epochs_no_improve = 0
        else:
            epochs_no_improve += 1

            # Check early stopping condition
            if epochs_no_improve == patience:
                print(f"Stopping after {epoch} epochs due to no improvement.")
                model.load_state_dict(best_model)
                break
        # Print progress at set epoch intervals if desired
        if print_progress and (epoch + 1) % 10 == 0:
            print(f"Epoch {epoch+1} Train Loss: {train_epoch_loss:.4}, ", end="")
            print(f"Val Loss: {val_epoch_loss:.4}")

    return train_loss, val_loss


def weight_reset(m):
    """Reset all weights in an NN."""
    reset_parameters = getattr(m, "reset_parameters", None)
    if callable(reset_parameters):
        m.reset_parameters()

In [39]:
# ------------------- Cross-validation -------------------------------------- #


def kfold_cv_train(
    dataset,
    model,
    optimizer,
    loss_fn=nn.MSELoss(),
    n_splits=5,
    batch_size=128,
    n_epochs=2000,
    patience=100,
    shuffle=True,
    rng=SEED,
):
    """Train a NN with K-Fold cross-validation."""
    kfold = KFold(n_splits=n_splits, shuffle=shuffle, random_state=rng)
    best_losses = []

    for fold, (train_ids, val_ids) in enumerate(kfold.split(dataset)):

        # Print
        print(f"FOLD {fold}")
        print("--------------------------------")

        # Sample elements randomly from a given list of ids, no replacement.
        train_subsampler = torch.utils.data.SubsetRandomSampler(train_ids)
        val_subsampler = torch.utils.data.SubsetRandomSampler(val_ids)

        # Define data loaders for training and testing data in this fold
        train_loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, sampler=train_subsampler)
        val_loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, sampler=val_subsampler)

        # Train
        train_loss, val_loss = train_network(
            model=model,
            train_data=train_loader,
            val_data=val_loader,
            optimizer=optimizer,
            loss_fn=loss_fn,
            n_epochs=n_epochs,
            patience=patience,
            print_progress=False,
        )
        best_losses.append(min(val_loss))
        model.apply(weight_reset)

    return sum(best_losses) / n_splits, train_loss, val_loss

In [ ]:
# ------------------- Bayesian optimization --------------------------------- #


class tqdm_skopt(object):
    """Progress bar object for functions with callbacks."""

    def __init__(self, **kwargs):
        self._bar = tqdm(**kwargs)

    def __call__(self, res):
        """Update bar with intermediate results."""
        self._bar.update()


# Set parameter search space
# sourcery skip: merge-list-append
space = []
space.append(Real(1e-5, 1e-1, name="learning_rate"))
space.append(Real(1e-10, 1e-1, name="regularization"))
space.append(Integer(16, 128, name="conv1_channels"))
space.append(Integer(16, 128, name="conv2_channels"))
space.append(Integer(16, 128, name="linear1_neurons"))
space.append(Integer(16, 128, name="linear2_neurons"))

# Set default hyperparameters
default_params = [1e-3, 1e-5, 32, 64, 64, 32]

batch_size = 128
activation = nn.ReLU()

# Work in progress
@use_named_args(dimensions=space)
def fitness(learning_rate, regularization, conv1_channels, conv2_channels, linear1_neurons, linear2_neurons):
    """Perform Bayesian Hyperparameter tuning."""

    model = CNN(
        conv1_channels=conv1_channels,
        conv2_channels=conv2_channels,
        linear1_neurons=linear1_neurons,
        linear2_neurons=linear2_neurons,
    )
    model.to(device)
    optimizer = optim.AdamW(model.parameters(), lr=learning_rate, weight_decay=regularization)
    # Create k-fold cross validation
    avg_best_loss, *_ = kfold_cv_train(dataset=kfold_data, model=model, optimizer=optimizer, batch_size=batch_size)
    # print(f'Avg. best validation loss: {sum(best_losses)/n_splits}')

    return avg_best_loss


n_calls = 100
# Hyperparemeter search using Gaussian process minimization
gp_result = gp_minimize(
    func=fitness,
    x0=default_params,
    dimensions=space,
    n_calls=n_calls,
    random_state=SEED,
    verbose=True,
    callback=[tqdm_skopt(total=n_calls, desc="Gaussian Process")],
)

plot_convergence(gp_result)
plot_objective(gp_result)
plot_evaluations(gp_result)
gp_result.x





Gaussian Process:   0%|          | 0/100 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/torch/nn/modules/lazy.py:178: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '


Iteration No: 1 started. Evaluating function at provided point.
FOLD 0
--------------------------------
Stopping after 105 epochs due to no improvement.
FOLD 1
--------------------------------
